<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/master/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Striving for Simplicity: The All Convolutional Net
###### *Wei-Tse Yang, Sunwei Wang, and Qingyuan Cao*
###### 14/4/2020
In this notebook, we try to reproduce the TABLE 3 in [original paper](https://arxiv.org/abs/1412.6806). The source code of one of the models on ```Pytorch``` and the training procedure on Google Colab can be found in [github](https://github.com/StefOe/all-conv-pytorch). **We adopt the original training procedure and change it to the Python class. Also, we build the models from scratch on Pytorch.**

---

# Brief Introduction
The paper shows that replacing the max-pooling with the convolutional with increased strides can improve the performance. The authors prove it by training models with max-pooling, models removing max-pooling, and models replacing max-pooling with convolutional layers. The results show the models replacing max-pooling with convolutional layers with strides generally have better performance. We provide the detail explanation as follows. The authors tested 12 networks by designing 3 model bases and 3 branches. 

## *Base: Model A, Model B, and Model C*
Since the design of convolutional layers would influence the performance, the authors test three model bases. **Model A** uses the 5x5 strides. **Model B** uses the 5x5 strides but also adds one convolutional layer with 1x1 strides after that. **Model C** uses two convolutional layers with 3x3 strides. 

## *Branch: Model, Strided-CNN, ConvPool-CNN, and ALL-CNN*
Each model base has one original model and thee branches. **“Model”** is the model with max-pooling. **“Strided-CNN”** is the model removing max-pooling. **“All-CNN”** is the model replacing max-pooling with convolutional strides. The better performance of “All-CNN” might result from more parameters than “Model” and “Strided-CNN”. To solve it, “ConvPool-CNN” is proposed. **“ConvPool-CNN”** is the model with max-pooling and one more convolutional layer before the pooling. “ConvPool-CNN” should have the same number of parameters as “All-CNN”. Therefore, if “All-CNN” has a better performance than “ConvPool-CNN”, we can prove the better performance on “All-CNN” does not result from more parameters. 


---
# Experiment Setup 
All 12 networks are trained on the CIFAR-10 with the stochastic gradient descent with a fixed momentum of 0.9 and 350 epochs. The learning rate γ is chosen from the set ∈ [0.25, 0.1, 0.05, 0.01]. It is also scheduled by multiplying with a fixed factor of 0.1 in the epoch S= [200, 250, 300]. The paper only presents the best performance among all learning rates. Based on the source code, the performance is directly evaluated on the CIFAR-10 test set. **The source code did not use the cross-validation for hyper-parameter tuning!** 

---

# Results


| Model      |  Error Rate of Paper  | Error Rate of Ours| 
|-|-|-|
|   Model A   | 12.47%|19.27%|
|   Strided-CNN-A  |13.46% |20.27%|
|   **ConvPool-CNN-A**  |**10.21%**|**15.46%**|
|   ALL-CNN-A  |10.30% |15.60%|
| | ||
|   Model B | 10.20%|**17.01%**|
|   Strided-CNN-B  | 10.98%|23.20%|
|   ConvPool-CNN-B  | 9.33%|18.22%|
|   ALL-CNN-B   | ***9.10%***|*29.48%|
| | ||
|   Model C  | 9.74%|?|
|   Strided-CNN-C  | 10.19%|?|
|   ConvPool-CNN-C  | 9.31%|?|
|   ALL-CNN-C  | ***9.08%***|?|



In [0]:
# Copy the trianing chunk to here?

# Cross-Valudation 

In [0]:
# Copy the trianing chunk to here?

# DropOut and Batch Normalization
Add motivation and description. 

In [0]:
# Copy the trianing chunk to here?

# Optimizer
Add motivation and description. 

In [0]:
# Copy the trianing chunk to here?

# Softmax?

In [0]:
# Copy the trianing chunk to here?

# Appendix


*   [Github Repository](https://github.com/wtsyang/dl-reproducibility-project)
*   [A Python Class to build all Models](https://github.com/wtsyang/dl-reproducibility-project/blob/master/Model.py)
*   [A Python Class for Training Procedure](https://github.com/wtsyang/dl-reproducibility-project/blob/master/Training.py)
*   [The Notebooks for Model A](https://github.com/wtsyang/dl-reproducibility-project/tree/master/modelA)








